In [1]:
#import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set your executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\mrale\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
#, then set up the URL for scraping
#visit the mars nasa news site
url = 'http:redplanetscience.com/'
browser.visit(url)

#optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

#ABOVE CODE
#code accomplishes 2 things:
# 1. we're searching for elements with a specific combination of tag (div) and attribute (list_text).
# 2. we're also telling our browser to wait one second before searching for components. 
# the optional delay is useful because sometimes dynamic pages take a little while to load, especially if theya re image-heavy

True

In [4]:
#set up the html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

# Notice how we've assigned slide_elem as the variable to look for the <div /> tag and its descendent (the other tags within
# the <div /> element)? This is our parent element. This means that this element holds all of the other elements within it, 
# and we'll reference it when we want to filter search results even further. The . is used for selecting classes, such as 
# list_text, so the code 'div.list_text' pinpoints the <div /> tag with the class of list_text. CSS works from right to left,
# such as returning the last item on the list instead of the first. Because of this, when using select_one, the first 
# matching element returned will be a <li /> element with a class of slide and all nested elements within it.

In [5]:
#Robin wants to collect the most recent news artcile along with its summary

#assign the title and summary text to variables we'll reference later
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Wins 4 Webbys, 4 People's Voice Awards</div>

In [6]:
# use the parent element to find the first 'a' tag and save it as 'news_title' 
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

#ABOVE CODE
# .get_text() only the text of the element is returned

"NASA Wins 4 Webbys, 4 People's Voice Awards"

In [7]:
# use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA\'s Global Climate Change website and Solar System Interactive.'

### Image Scraping

In [8]:
#visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# find the click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
#parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
#find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

#CODE ABOVE
# an img tag is nested within this HTML, so we've included it.
# .get('src') pulls the link to the image

#what we've done here is tell BeautifulSoup to look inside the <img /> tag for an image with a class of fancybox-image
# Basically we're saying, "this is where the image we want lives - use the link that's inside these tags"


'image/featured/mars3.jpg'

In [12]:
#use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [13]:
#scrape the entire table with Pandas

# creating a DataFrame from the HTML table.
# the Pandas function read_html() specifically searches for and returns a list of tables found in the HTMl.
#by specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or teh first item in the list.
df = pd.read_html('https://galaxyfacts-mars.com')[0]

#we assign columns to the new DataFrame for additional clarity
df.columns=['description', 'Mars', 'Earth']

#by using .set_index() function, we're turning the description column into the DataFrame's index.
#inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a new variable.
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
#convert our DF back into HTML-ready code using the .to_html() function.
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
#end the automated browsing session.
browser.quit()